In [1]:
strategy_name = "IC"
verbose = True
plot_graph = False
recent_data_only=True

In [2]:
%load_ext autoreload
%autoreload 2

# equity_index_future_basis_analysis
股指期货基差套利分析

## 背景原理

### 关于股指期货

[https://wiki.mbalib.com/wiki/%E8%82%A1%E6%8C%87%E6%9C%9F%E8%B4%A7](https://wiki.mbalib.com/wiki/%E8%82%A1%E6%8C%87%E6%9C%9F%E8%B4%A7)

股指期货相当于对股票指数中所包含的一篮子股票打包作为期货进行买卖的概念。买入

### 关于贴水和升水

股指期货与现货指数价格的差被称为基差，当股指期货价格高于现货指数价格时，股指期货处于升水，基差为正;反之，股指期货处于贴水，基差为负。

### 关于股指期货贴水获利的原理

[https://xueqiu.com/1445029417/143207882](https://xueqiu.com/1445029417/143207882)



## 回测分析

回测的目的是：
1. 确认在过去的回测期间内，策略是有效可盈利的。
2. 在策略的大框架下选择标的证券。

### 安装环境依赖

### 策略1

最基础的策略为，选定一个标的合约，并不断展期持有。例如持有当月合约，在到期时展期至下一时间的合约。例如在4月份持有 IC2104，并在4月交割日展期为 IC2105。

在这个策略中可选参数为：
1. 合约的期限。当月、下月、当季、下季。
2. 标的。IC、IF、IH

回测后发现，持有下月的 IC 合约，基差获利最多。

### 策略2

在策略1的基础上，每次展期所获得的基差收益来源于两个合约的基差之差。而基差之差是会浮动变化的，具体表现为：

在上涨预期强或者上涨趋势中，基差缩小，甚至部分时候会出现升水。
在震荡预期中，基差保持不变。
在下跌预期中，基差扩大。

利用这个特性，可以对策略1进行优化：
1. 在过去N个交易日中，如果是上涨趋势，则暂缓展期，回退至持有当月合约的策略。
2. 在过去N个交易日中，如果是震荡预期或下跌预期，则持有次月合约，次月合约转当月合约时进行展期。

展期本质上是走某个时间点，将时间价值兑换为对应的基差价值。而对应时点的基差价值是由当时的做空成本决定的，择时展期本质上就是对做空成本的择时。

具体策略表现为，计算每个合约的年化基差率，并将年化基差率作为时间序列进行计算，同时生成技术指标布林通道，布林通道参数为 120 日均值，0.5 标准差。当目前的年化基差率大于布林通道上轨时进行展期。2017年至今回测可以提高20%收益率。

In [10]:
from spread_rolling import SpreadRollingStrategyBackTestingWrapper
from market_data.data_definition import ic_daily_tick_data, ic_index_data
from datetime import datetime

s1 = SpreadRollingStrategyBackTestingWrapper(future_data=ic_daily_tick_data, underlying_data=ic_index_data)
s1.download_data()
s1.back_test_and_get_today_target_pos(start_date=datetime(2019,1,1))

Saving IC2112
Saving IC2201
Saving IC2203
Saving IC2206
Saving IF2112
Saving IF2201
Saving IF2203
Saving IF2206
Saving IH2112
Saving IH2201
Saving IH2203
Saving IH2206
Saving T2112
Saving T2203
Saving T2206
Saving TF2112
Saving TF2203
Saving TF2206
Saving TS2112
Saving TS2203
Saving TS2206
Saving T99
Saving TS99
Saving IH99
Saving IC99
Saving IF99
Saving TF99
2021-12-02 14:53:59.735436	开始加载历史数据
2021-12-02 14:53:59.737795	000905.SSE加载进度： [3%]
2021-12-02 14:53:59.740661	000905.SSE加载进度： [6%]
2021-12-02 14:53:59.743235	000905.SSE加载进度： [8%]
2021-12-02 14:53:59.745162	000905.SSE加载进度：# [11%]
2021-12-02 14:53:59.747693	000905.SSE加载进度：# [14%]
2021-12-02 14:53:59.752777	000905.SSE加载进度：# [17%]
2021-12-02 14:53:59.757827	000905.SSE加载进度：# [20%]
2021-12-02 14:53:59.760846	000905.SSE加载进度：## [23%]
2021-12-02 14:53:59.763622	000905.SSE加载进度：## [25%]
2021-12-02 14:53:59.766846	000905.SSE加载进度：## [28%]
2021-12-02 14:53:59.768913	000905.SSE加载进度：### [31%]
2021-12-02 14:53:59.771458	000905.SSE加载进度：### [34%]
2

2021-12-02 14:53:59.941635	IC1606.CFFEX加载进度：###### [68%]
2021-12-02 14:53:59.942978	IC1606.CFFEX加载进度：####### [70%]
2021-12-02 14:53:59.943690	IC1606.CFFEX加载进度：####### [73%]
2021-12-02 14:53:59.944295	IC1606.CFFEX加载进度：####### [76%]
2021-12-02 14:53:59.944843	IC1606.CFFEX加载进度：####### [79%]
2021-12-02 14:53:59.945447	IC1606.CFFEX加载进度：######## [82%]
2021-12-02 14:53:59.945990	IC1606.CFFEX加载进度：######## [84%]
2021-12-02 14:53:59.946668	IC1606.CFFEX加载进度：######## [87%]
2021-12-02 14:53:59.947248	IC1606.CFFEX加载进度：######### [90%]
2021-12-02 14:53:59.947789	IC1606.CFFEX加载进度：######### [93%]
2021-12-02 14:53:59.948385	IC1606.CFFEX加载进度：######### [96%]
2021-12-02 14:53:59.948931	IC1606.CFFEX加载进度：######### [98%]
2021-12-02 14:53:59.948987	IC1606.CFFEX历史数据加载完成，数据量：0
2021-12-02 14:53:59.949629	IC1607.CFFEX加载进度： [3%]
2021-12-02 14:53:59.950578	IC1607.CFFEX加载进度： [6%]
2021-12-02 14:53:59.951317	IC1607.CFFEX加载进度： [8%]
2021-12-02 14:53:59.951862	IC1607.CFFEX加载进度：# [11%]
2021-12-02 14:53:59.952463	IC1607.CFFE

2021-12-02 14:54:00.170978	IC1702.CFFEX加载进度：# [17%]
2021-12-02 14:54:00.171819	IC1702.CFFEX加载进度：# [20%]
2021-12-02 14:54:00.172835	IC1702.CFFEX加载进度：## [23%]
2021-12-02 14:54:00.173761	IC1702.CFFEX加载进度：## [25%]
2021-12-02 14:54:00.175030	IC1702.CFFEX加载进度：## [28%]
2021-12-02 14:54:00.175944	IC1702.CFFEX加载进度：### [31%]
2021-12-02 14:54:00.176758	IC1702.CFFEX加载进度：### [34%]
2021-12-02 14:54:00.177764	IC1702.CFFEX加载进度：### [37%]
2021-12-02 14:54:00.180109	IC1702.CFFEX加载进度：### [39%]
2021-12-02 14:54:00.180793	IC1702.CFFEX加载进度：#### [42%]
2021-12-02 14:54:00.181635	IC1702.CFFEX加载进度：#### [45%]
2021-12-02 14:54:00.182628	IC1702.CFFEX加载进度：#### [48%]
2021-12-02 14:54:00.183674	IC1702.CFFEX加载进度：##### [51%]
2021-12-02 14:54:00.185235	IC1702.CFFEX加载进度：##### [53%]
2021-12-02 14:54:00.185911	IC1702.CFFEX加载进度：##### [56%]
2021-12-02 14:54:00.187109	IC1702.CFFEX加载进度：##### [59%]
2021-12-02 14:54:00.188275	IC1702.CFFEX加载进度：###### [62%]
2021-12-02 14:54:00.190420	IC1702.CFFEX加载进度：###### [65%]
2021-12-02 14:54:0

2021-12-02 14:54:00.380219	IC1709.CFFEX加载进度：## [25%]
2021-12-02 14:54:00.380896	IC1709.CFFEX加载进度：## [28%]
2021-12-02 14:54:00.381543	IC1709.CFFEX加载进度：### [31%]
2021-12-02 14:54:00.382202	IC1709.CFFEX加载进度：### [34%]
2021-12-02 14:54:00.382848	IC1709.CFFEX加载进度：### [37%]
2021-12-02 14:54:00.383491	IC1709.CFFEX加载进度：### [39%]
2021-12-02 14:54:00.384033	IC1709.CFFEX加载进度：#### [42%]
2021-12-02 14:54:00.384723	IC1709.CFFEX加载进度：#### [45%]
2021-12-02 14:54:00.386467	IC1709.CFFEX加载进度：#### [48%]
2021-12-02 14:54:00.387039	IC1709.CFFEX加载进度：##### [51%]
2021-12-02 14:54:00.387876	IC1709.CFFEX加载进度：##### [53%]
2021-12-02 14:54:00.389133	IC1709.CFFEX加载进度：##### [56%]
2021-12-02 14:54:00.390350	IC1709.CFFEX加载进度：##### [59%]
2021-12-02 14:54:00.391550	IC1709.CFFEX加载进度：###### [62%]
2021-12-02 14:54:00.392414	IC1709.CFFEX加载进度：###### [65%]
2021-12-02 14:54:00.394444	IC1709.CFFEX加载进度：###### [68%]
2021-12-02 14:54:00.395289	IC1709.CFFEX加载进度：####### [70%]
2021-12-02 14:54:00.396203	IC1709.CFFEX加载进度：####### [73%]
20

2021-12-02 14:54:00.595980	IC1804.CFFEX加载进度：##### [59%]
2021-12-02 14:54:00.596728	IC1804.CFFEX加载进度：###### [62%]
2021-12-02 14:54:00.597487	IC1804.CFFEX加载进度：###### [65%]
2021-12-02 14:54:00.598192	IC1804.CFFEX加载进度：###### [68%]
2021-12-02 14:54:00.598777	IC1804.CFFEX加载进度：####### [70%]
2021-12-02 14:54:00.599387	IC1804.CFFEX加载进度：####### [73%]
2021-12-02 14:54:00.599947	IC1804.CFFEX加载进度：####### [76%]
2021-12-02 14:54:00.600537	IC1804.CFFEX加载进度：####### [79%]
2021-12-02 14:54:00.601323	IC1804.CFFEX加载进度：######## [82%]
2021-12-02 14:54:00.601929	IC1804.CFFEX加载进度：######## [84%]
2021-12-02 14:54:00.602533	IC1804.CFFEX加载进度：######## [87%]
2021-12-02 14:54:00.603103	IC1804.CFFEX加载进度：######### [90%]
2021-12-02 14:54:00.603694	IC1804.CFFEX加载进度：######### [93%]
2021-12-02 14:54:00.604417	IC1804.CFFEX加载进度：######### [96%]
2021-12-02 14:54:00.605118	IC1804.CFFEX加载进度：######### [98%]
2021-12-02 14:54:00.605189	IC1804.CFFEX历史数据加载完成，数据量：0
2021-12-02 14:54:00.606425	IC1805.CFFEX加载进度： [3%]
2021-12-02 14:54:00.

...

2021-12-02 14:54:01.890673	IC2105.CFFEX加载进度：### [39%]
2021-12-02 14:54:01.891466	IC2105.CFFEX加载进度：#### [42%]
2021-12-02 14:54:01.892243	IC2105.CFFEX加载进度：#### [45%]
2021-12-02 14:54:01.893603	IC2105.CFFEX加载进度：#### [48%]
2021-12-02 14:54:01.894651	IC2105.CFFEX加载进度：##### [51%]
2021-12-02 14:54:01.895824	IC2105.CFFEX加载进度：##### [53%]
2021-12-02 14:54:01.897126	IC2105.CFFEX加载进度：##### [56%]
2021-12-02 14:54:01.898479	IC2105.CFFEX加载进度：##### [59%]
2021-12-02 14:54:01.899908	IC2105.CFFEX加载进度：###### [62%]
2021-12-02 14:54:01.901472	IC2105.CFFEX加载进度：###### [65%]
2021-12-02 14:54:01.902578	IC2105.CFFEX加载进度：###### [68%]
2021-12-02 14:54:01.904794	IC2105.CFFEX加载进度：####### [70%]
2021-12-02 14:54:01.905747	IC2105.CFFEX加载进度：####### [73%]
2021-12-02 14:54:01.909392	IC2105.CFFEX加载进度：####### [76%]
2021-12-02 14:54:01.912818	IC2105.CFFEX加载进度：####### [79%]
2021-12-02 14:54:01.914248	IC2105.CFFEX加载进度：######## [82%]
2021-12-02 14:54:01.915249	IC2105.CFFEX加载进度：######## [84%]
2021-12-02 14:54:01.916011	IC2105.CF

2021-12-02 14:54:02.105699	IC2110.CFFEX加载进度：####### [73%]
2021-12-02 14:54:02.106366	IC2110.CFFEX加载进度：####### [76%]
2021-12-02 14:54:02.107236	IC2110.CFFEX加载进度：####### [79%]
2021-12-02 14:54:02.107957	IC2110.CFFEX加载进度：######## [82%]
2021-12-02 14:54:02.108813	IC2110.CFFEX加载进度：######## [84%]
2021-12-02 14:54:02.109584	IC2110.CFFEX加载进度：######## [87%]
2021-12-02 14:54:02.112401	IC2110.CFFEX加载进度：######### [90%]
2021-12-02 14:54:02.114271	IC2110.CFFEX加载进度：######### [93%]
2021-12-02 14:54:02.114947	IC2110.CFFEX加载进度：######### [96%]
2021-12-02 14:54:02.115647	IC2110.CFFEX加载进度：######### [98%]
2021-12-02 14:54:02.115707	IC2110.CFFEX历史数据加载完成，数据量：33
2021-12-02 14:54:02.116574	IC2111.CFFEX加载进度： [3%]
2021-12-02 14:54:02.117315	IC2111.CFFEX加载进度： [6%]
2021-12-02 14:54:02.118037	IC2111.CFFEX加载进度： [8%]
2021-12-02 14:54:02.118847	IC2111.CFFEX加载进度：# [11%]
2021-12-02 14:54:02.119639	IC2111.CFFEX加载进度：# [14%]
2021-12-02 14:54:02.120365	IC2111.CFFEX加载进度：# [17%]
2021-12-02 14:54:02.120952	IC2111.CFFEX加载进度：# [2

2021-12-02 14:54:02.325123	IC2204.CFFEX加载进度：###### [62%]
2021-12-02 14:54:02.325743	IC2204.CFFEX加载进度：###### [65%]
2021-12-02 14:54:02.326586	IC2204.CFFEX加载进度：###### [68%]
2021-12-02 14:54:02.327450	IC2204.CFFEX加载进度：####### [70%]
2021-12-02 14:54:02.328188	IC2204.CFFEX加载进度：####### [73%]
2021-12-02 14:54:02.328992	IC2204.CFFEX加载进度：####### [76%]
2021-12-02 14:54:02.329848	IC2204.CFFEX加载进度：####### [79%]
2021-12-02 14:54:02.330632	IC2204.CFFEX加载进度：######## [82%]
2021-12-02 14:54:02.331445	IC2204.CFFEX加载进度：######## [84%]
2021-12-02 14:54:02.332503	IC2204.CFFEX加载进度：######## [87%]
2021-12-02 14:54:02.333518	IC2204.CFFEX加载进度：######### [90%]
2021-12-02 14:54:02.334311	IC2204.CFFEX加载进度：######### [93%]
2021-12-02 14:54:02.335165	IC2204.CFFEX加载进度：######### [96%]
2021-12-02 14:54:02.335840	IC2204.CFFEX加载进度：######### [98%]
2021-12-02 14:54:02.335894	IC2204.CFFEX历史数据加载完成，数据量：0
2021-12-02 14:54:02.336781	IC2205.CFFEX加载进度： [3%]
2021-12-02 14:54:02.337662	IC2205.CFFEX加载进度： [6%]
2021-12-02 14:54:02.338403

2021-12-02 14:54:02.554737	IC2212.CFFEX加载进度：###### [65%]
2021-12-02 14:54:02.555707	IC2212.CFFEX加载进度：###### [68%]
2021-12-02 14:54:02.556571	IC2212.CFFEX加载进度：####### [70%]
2021-12-02 14:54:02.557479	IC2212.CFFEX加载进度：####### [73%]
2021-12-02 14:54:02.558154	IC2212.CFFEX加载进度：####### [76%]
2021-12-02 14:54:02.558992	IC2212.CFFEX加载进度：####### [79%]
2021-12-02 14:54:02.560076	IC2212.CFFEX加载进度：######## [82%]
2021-12-02 14:54:02.561134	IC2212.CFFEX加载进度：######## [84%]
2021-12-02 14:54:02.562324	IC2212.CFFEX加载进度：######## [87%]
2021-12-02 14:54:02.563476	IC2212.CFFEX加载进度：######### [90%]
2021-12-02 14:54:02.564243	IC2212.CFFEX加载进度：######### [93%]
2021-12-02 14:54:02.565602	IC2212.CFFEX加载进度：######### [96%]
2021-12-02 14:54:02.566364	IC2212.CFFEX加载进度：######### [98%]
2021-12-02 14:54:02.566422	IC2212.CFFEX历史数据加载完成，数据量：0
2021-12-02 14:54:02.566452	所有历史数据加载完成
2021-12-02 14:54:02.567057	策略初始化完成
2021-12-02 14:54:02.567161	开始回放历史数据
2019-01-04 00:00:00+00:00 basis_map {
  "IC1901.CFFEX": 1.7792563817038343

2021-12-02 14:54:03.291227	历史数据回放结束


In [11]:
s1.get_latest_pos_date()

datetime.date(2021, 12, 2)

In [12]:
s1.show_result()

2021-12-02 14:54:11.502273	开始计算逐日盯市盈亏
2021-12-02 14:54:11.541701	逐日盯市盈亏计算完成
2021-12-02 14:54:11.541828	开始计算策略统计指标
2021-12-02 14:54:11.548029	------------------------------
2021-12-02 14:54:11.548228	首个交易日：	2019-01-03
2021-12-02 14:54:11.548397	最后交易日：	2021-12-02
2021-12-02 14:54:11.548425	总交易日：	708
2021-12-02 14:54:11.548442	盈利交易日：	386
2021-12-02 14:54:11.548469	亏损交易日：	318
2021-12-02 14:54:11.548493	起始资金：	1,000,000.00
2021-12-02 14:54:11.548529	结束资金：	2,474,383.72
2021-12-02 14:54:11.548559	总收益率：	147.44%
2021-12-02 14:54:11.548635	年化收益：	49.98%
2021-12-02 14:54:11.548689	最大回撤: 	-413,400.00
2021-12-02 14:54:11.548778	百分比最大回撤: -26.37%
2021-12-02 14:54:11.548806	最长回撤天数: 	63
2021-12-02 14:54:11.549286	总盈亏：	1,474,383.72
2021-12-02 14:54:11.549402	总手续费：	1,916.28
2021-12-02 14:54:11.549516	总滑点：	0.00
2021-12-02 14:54:11.549570	总成交金额：	83,316,660.00
2021-12-02 14:54:11.549597	总成交笔数：	49
2021-12-02 14:54:11.549614	日均盈亏：	2,082.46
2021-12-02 14:54:11.549685	日均手续费：	2.71
2021-12-02 14:54:11.549735	日均滑点：	